In [178]:
#r "nuget:FParsec"
open FParsec

let loadFile fname = System.IO.File.ReadAllText(fname)

let sample = "day5.sample.txt"
let data = "day5.data.txt"



Installed Packages FParsec, 1.1.1

In [179]:
// Parser... I may have over complicated this...

let str = pstring
let ws = spaces
type UserState = unit
type Parser<'t> = Parser<'t, UserState>


type parsedData = {
    Seeds : int64 list
    SeedToSoilMap : (int64 * int64 * int64) list
    SoilToFertilizerMap : (int64 * int64 * int64) list
    FertilizerToWaterMap : (int64 * int64 * int64) list
    WaterToLightMap : (int64 * int64 * int64) list
    LightToTemperatureMap : (int64 * int64 * int64) list
    TemperatureToHumidityMap : (int64 * int64 * int64) list
    HumidityToLocationMap : (int64 * int64 * int64) list
}

let pRecord p1 p2 p3 p4 p5 p6 p7 p8 =
    pipe5 p1 p2 p3 p4 (tuple4 p5 p6 p7 p8) (fun p1 p2 p3 p4 (p5, p6, p7, p8) ->
        {
            Seeds=p1
            SeedToSoilMap=p2
            SoilToFertilizerMap=p3
            FertilizerToWaterMap=p4
            WaterToLightMap=p5
            LightToTemperatureMap=p6
            TemperatureToHumidityMap=p7
            HumidityToLocationMap=p8
        }
        )

let seedsParser = str "seeds:" >>. ws >>. manyTill ( pint64 .>> ws ) (str "seed-to-soil map:")
let mapRangeParser : Parser<_> =
    tuple3 (pint64 .>> ws) (pint64 .>> ws) (pint64 .>> ws)
let inputParser : Parser<_> =
    pRecord
        (seedsParser .>> ws)
        (manyTill mapRangeParser (str "soil-to-fertilizer map:") .>> ws)
        (manyTill mapRangeParser (str "fertilizer-to-water map:") .>> ws)
        (manyTill mapRangeParser (str "water-to-light map:") .>> ws)
        (manyTill mapRangeParser (str "light-to-temperature map:") .>> ws)
        (manyTill mapRangeParser (str "temperature-to-humidity map:") .>> ws)
        (manyTill mapRangeParser (str "humidity-to-location map:") .>> ws)
        (many mapRangeParser)

let parseInput (s: string) =
    match run inputParser s with
    | Success (result, _, _) -> result
    | Failure (message, _, _) -> failwith message

// test parser on sample
parseInput (loadFile sample) |> printfn "%A"




{ Seeds = [79L; 14L; 55L; 13L]
  SeedToSoilMap = [(50L, 98L, 2L); (52L, 50L, 48L)]
  SoilToFertilizerMap = [(0L, 15L, 37L); (37L, 52L, 2L); (39L, 0L, 15L)]
  FertilizerToWaterMap =
   [(49L, 53L, 8L); (0L, 11L, 42L); (42L, 0L, 7L); (57L, 7L, 4L)]
  WaterToLightMap = [(88L, 18L, 7L); (18L, 25L, 70L)]
  LightToTemperatureMap = [(45L, 77L, 23L); (81L, 45L, 19L); (68L, 64L, 13L)]
  TemperatureToHumidityMap = [(0L, 69L, 1L); (1L, 0L, 69L)]
  HumidityToLocationMap = [(60L, 56L, 37L); (56L, 93L, 4L)] }


In [180]:
// Range mapper inner logic

type RangeDescription = {
    fromStart : int
    toStart : int
    count : int
    }

type RangeMap = int64 -> int64

module RangeMap = 
    let create (rds : (int64 * int64 * int64) list) : RangeMap =
        fun inValue -> 
            rds
            |> List.tryFind (fun (_, fromStart, count) ->
                fromStart <= inValue
                && inValue < fromStart + count
            )
            |> Option.map (fun (toStart, fromStart, _) ->
                inValue + (toStart - fromStart)
            )
            |> Option.defaultValue inValue

let testRangeDescriptions = [
    (1L, 10L, 5L)
    (40L, 30L, 10L)
]

let testMap = RangeMap.create testRangeDescriptions

testMap 12


3

In [181]:
type Seed = Seed of int64

type Soil = Soil of int64
    
type Fertilizer = Fertilizer of int64

type Water = Water of int64

type Light = Light of int64

type Temperature = Temperature of int64

type Humidity = Humidity of int64

type Location = Location of int64

type Mappers = {
    SeedToSoil : Seed -> Soil
    SoilToFertilizer : Soil -> Fertilizer
    FertilizerToWater : Fertilizer -> Water
    WaterToLight : Water -> Light
    LightToTemperature : Light -> Temperature
    TemperatureToHumidity : Temperature -> Humidity
    HumidityToLocation : Humidity -> Location
}

module Mappers =

    let private seedsToSoil rangeMap (Seed x) : Soil = 
        x |> rangeMap |> Soil

    let private soilToFertilizer rangeMap (Soil x) : Fertilizer =
        x |> rangeMap |> Fertilizer

    let private fertilizerToWater rangeMap (Fertilizer x) : Water =
        x |> rangeMap |> Water

    let private waterToLight rangeMap (Water x) : Light =
        x |> rangeMap |> Light

    let private lightToTemperature rangeMap (Light x) : Temperature =
        x |> rangeMap |> Temperature
    
    let private temperatureToHumidity rangeMap (Temperature x) : Humidity =
        x |> rangeMap |> Humidity
    
    let private humidityToLocation rangeMap (Humidity x) : Location =
        x |> rangeMap |> Location

    let private tee x =
        printfn "%A" x
        x

    let create parsedData : Mappers =
        let pd = parsedData
        let rm = RangeMap.create
        {
            SeedToSoil = pd.SeedToSoilMap |> rm |> seedsToSoil
            SoilToFertilizer = pd.SoilToFertilizerMap |> rm |> soilToFertilizer
            FertilizerToWater = pd.FertilizerToWaterMap |> rm |> fertilizerToWater
            WaterToLight = pd.WaterToLightMap |> rm |> waterToLight
            LightToTemperature = pd.LightToTemperatureMap |> rm |> lightToTemperature
            TemperatureToHumidity = pd.TemperatureToHumidityMap |> rm |> temperatureToHumidity
            HumidityToLocation = pd.HumidityToLocationMap |> rm |> humidityToLocation
        }

    let seedToLocation (mappers : Mappers) (seed : Seed) : Location = 
        seed //|> tee
        |> mappers.SeedToSoil //|> tee
        |> mappers.SoilToFertilizer //|> tee
        |> mappers.FertilizerToWater //|> tee
        |> mappers.WaterToLight //|> tee
        |> mappers.LightToTemperature //|> tee
        |> mappers.TemperatureToHumidity //|> tee
        |> mappers.HumidityToLocation //|> tee
        


In [182]:
// time to test the sample data

let parsedData = loadFile sample |> parseInput
let mappers = Mappers.create parsedData
let seedToLocation = Mappers.seedToLocation mappers

display "Part 1 Sample data result"
parsedData.Seeds
|> List.map (Seed >> seedToLocation)
|> List.minBy (fun (Location x) -> x)


Part 1 Sample data result

Item,35


In [183]:
// Holy Canoly this had better work

let parsedData = loadFile data |> parseInput
let mappers = Mappers.create parsedData
let seedToLocation = Mappers.seedToLocation mappers

display "Part 1 Real data result"
parsedData.Seeds
|> List.map (Seed >> seedToLocation)
|> List.minBy (fun (Location x) -> x)

Part 1 Real data result

Item,331445006


In [184]:
let parsedData = loadFile sample |> parseInput
let mappers = Mappers.create parsedData
let seedToLocation = Mappers.seedToLocation mappers

let seedRanges = parsedData.Seeds |> List.chunkBySize 2
let seeds : int64 seq = seq {
        for seedRange in seedRanges do
            match seedRange with
            | [i; n] -> yield! {i..(i+n-1L)}
            | _ -> yield! Seq.empty
    }

display "Part 2 Sample data result"
seeds
|> Seq.map (Seed >> seedToLocation)
|> Seq.minBy (fun (Location x) -> x)

Part 2 Sample data result

Item,46


In [185]:
// Part 2 brute force... I don't think it's gonna work

// let parsedData = loadFile data |> parseInput
// let mappers = Mappers.create parsedData
// let seedToLocation = Mappers.seedToLocation mappers

// let seedRanges = parsedData.Seeds |> List.chunkBySize 2
// let seeds : int64 seq = seq {
//         for seedRange in seedRanges do
//             match seedRange with
//             | [i; n] -> yield! {i..(i+n-1L)}
//             | _ -> yield! Seq.empty
//     }

// display "Part 2 Real data result"
// seeds
// |> Seq.map (Seed >> seedToLocation)
// |> Seq.minBy (fun (Location x) -> x)

Part 2 Real data result

Error: Command cancelled.